# Title
[]()

In [1]:
from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [2]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
from pypdf import PdfReader
import pandas as pd
import openai
import os
from openai.embeddings_utils import get_embedding, cosine_similarity
import re
env_name = 'api_openai'

# Original Functions: `Vectorize_PDF` & `Chatbot`

In [ ]:
class Vectorize_PDF:
    
    def extract_text(self, pdf):
        print("Parsing paper")
        number_of_pages = len(pdf.pages)
        print(f"Total number of pages: {number_of_pages}")
        paper_text = []
        for i in range(number_of_pages):
            page = pdf.pages[i]
            page_text = []

            def visitor_body(text, cm, tm, fontDict, fontSize):
                x = tm[4]
                y = tm[5]
                # ignore header/footer
                if (y > 50 and y < 720) and (len(text.strip()) > 1):
                    page_text.append({
                    'fontsize': fontSize,
                    'text': text.strip().replace('\x03', ''),
                    'x': x,
                    'y': y
                    })

            _ = page.extract_text(visitor_text=visitor_body)

            blob_font_size = None
            blob_text = ''
            processed_text = []

            for t in page_text:
                if t['fontsize'] == blob_font_size:
                    blob_text += f" {t['text']}"
                    if len(blob_text) >= 2000:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                        blob_font_size = None
                        blob_text = ''
                else:
                    if blob_font_size is not None and len(blob_text) >= 1:
                        processed_text.append({
                            'fontsize': blob_font_size,
                            'text': blob_text,
                            'page': i
                        })
                    blob_font_size = t['fontsize']
                    blob_text = t['text']
                paper_text += processed_text
        print("Done parsing paper")
        return paper_text

    def create_df(self, pdf):
        print('Creating dataframe')
        filtered_pdf= []
        for row in pdf:
            if len(row['text']) < 130: # SH 2023-04-05 21:11 Increased from 30 to 130
                continue
            filtered_pdf.append(row)
        df = pd.DataFrame(filtered_pdf)
        # print(df.shape)
        # remove elements with identical df[text] and df[page] values
        df = df.drop_duplicates(subset=['text', 'page'], keep='first')
        df['length'] = df['text'].apply(lambda x: len(x))
        print('Done creating dataframe')
        return df
    
    def embeddings(self, df):
        print('Calculating embeddings')
        openai.api_key = os.getenv(env_name)
        embedding_model = "text-embedding-ada-002"
        embeddings = df.text.apply([lambda x: get_embedding(x, engine=embedding_model)])
        df["embeddings"] = embeddings
        print('Done calculating embeddings')
        return df

def process_pdf(pdf_filename, path, filename=None, append_version=False,
    save_path=r'C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\output\pickles'
    ):
    """
    Extract the text from the saved PDF and create the embeddings using OpenAI.
    Parameters:
        - pdf_filename (str): Filename of PDF.
        - path (raw string): File path copied from File Explorer.
    Returns:
        - DataFrame containing parsed text and word embeddings.
        - PdfReader(filename) class object.
        - Chatbot() class object.
    """
    print("Processing PDF")
    full_path = f'{path}/'.replace('\\','/')+pdf_filename

    pdf = PdfReader(full_path)
    chatbot = Vectorize_PDF()
    paper_text = chatbot.extract_text(pdf)
    df = chatbot.create_df(paper_text)
    df = chatbot.embeddings(df)
    if filename:
        try:
            filename = filename if (type(filename) == str) else (re.sub(r'(.*).pdf', r'\1', pdf_filename))
            savepickle(chatbot, filename='chatgpt_embeddings_'+filename, path=save_path)
            savepickle(df, filename='embeddings_df_'+filename, path=save_path)
        except:
            print('Unable to save outputs')
    
    return df, pdf, chatbot

class Chatbot:
    def __init__(self, system_role, model, temperature, n_choices, max_tokens):
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def search(self, df, query, n=3, pprint=True):
        query_embedding = get_embedding(
            query,
            engine="text-embedding-ada-002"
        )
        df["similarity"] = df.embeddings.apply(lambda x: cosine_similarity(x, query_embedding))
        
        results = df.sort_values("similarity", ascending=False, ignore_index=True)
        # make a dictionary of the the first three results with the page number as the key and the text as the value. The page number is a column in the dataframe.
        results = results.head(n)
        global sources 
        sources = []
        for i in range(n):
            # append the page number and the text as a dict to the sources list
            sources.append({'Page '+str(results.iloc[i]['page']): results.iloc[i]['text'][:150]+'...'})
        # print(sources)
        return results.head(n)
    
    def create_prompt(self, df, user_input):
        result = self.search(df, user_input, n=3)
        # print(result)
        system_role = f"""{self.system_role} You are given a query, 
        a series of text embeddings and the title from a paper in order of their cosine similarity to the query. 
        You must take the given embeddings and return a very detailed summary of the paper in the language of the query. 
        Here is the question: """+ user_input + """
            
        Here are the embeddings: 
            
            1.""" + str(result.iloc[0]['text']) + """
            2.""" + str(result.iloc[1]['text']) + """
            3.""" + str(result.iloc[2]['text']) + """
        """

        user_content = f"""Given the question: "{str(user_input)}". Return a detailed answer based on the paper:"""

        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_content},]

        print('Done creating prompt')
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response


def reply(df, user_input, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=None,
        system_role = """
        whose expertise is reading and summarizing scientific papers.
        """):
    chatbot = Chatbot(
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(
        df, user_input
        )
    qna = dict()
    qna['system_role'] = system_role
    qna['prompt'] = user_input
    try:
        response = chatbot.gpt(prompt)
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
    except:
        print('**Unable to get response**')
    return qna, chatbot


# Prompts

In [50]:
from itertools import product
task_part1 = [
    "Summarize the article",
    # "Summarize the article for a Tiktok post"
]
audience = [
    "lay audience",
    ""
    # "seniors"
]
task_part2 = [
    "",
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]
combinations = pd.DataFrame(product(task_part1, task_part2, audience), 
    columns=['task part 1', 'task part 2', 'audience'])


combinations['prompt'] = combinations.apply(
    lambda row: f"{row['task part 1']}{' for' if row['audience'] else ''} {row['audience']}. {row['task part 2']}", 
    axis=1)
combinations

,task part 1,task part 2,audience,prompt
0,Summarize the article,,lay audience,Summarize the article for lay audience.
1,Summarize the article,,,Summarize the article .


# Iteration 1: Discussion only

## Text to be used

In [4]:
# text_dict = {}
# text_dict[1] = {'title': 'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women'}

# text_dict[1]['text_discussion'] = """
# DISCUSSION
# This study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.

# Hypohydration and Pain in Women
# Independent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min after handgrip exercise (hereafter referred to as pain ratings) during ischemic pain stimulation, when compared with euhydration. These findings are consistent with those of previous studies that showed negative effects of mild hypohydration on experimental pain sensitivity in men (7, 8). Ogino et al. (7) observed a decrease in pain thresholds (i.e., increased pain sensitivity) following a combination of exercise-induced dehydration and fluid restriction, whereas Bear et al. (8) found an increase in pain intensity ratings but no difference in pain thresholds after 24 h of fluid restriction. Therefore, the current findings extend these previous results to women by showing that mild hypohydration also increases experimental pain sensitivity in women. The previous studies also did not include pain measures that reflect the affective/emotional experience of pain (i.e., pain tolerance or pain unpleasantness ratings) that may be important for chronic pain, whereas both pain threshold and pain intensity ratings assess the sensory/discriminative aspect of pain (49, 50). Although Bear et al. (8) measured pain tolerance, the nature of the stimulus-response relationship for the cold pressor task—i.e., plateau and gradual decline in the perceived intensity of the pain stimulus (41)—resulted in a large proportion (∼2/3) of participants who were able to tolerate the pain until the 4-min cut-off time, thereby precluding meaningful analyses of the results. This issue was not apparent with the ischemic pain test used in the present study, as there was only one instance where the 20-min time limit was reached. Therefore, the current findings further add to the previous data by showing that mild hypohydration also negatively impacts the affective/emotional aspect of pain, as evidenced by the decrease in pain tolerance and increase in pain unpleasantness ratings compared with EUH.

# The exact mechanisms underlying the hyperalgesic effect of hypohydration cannot be determined from the current data. However, one potential mechanism through which hypohydration may increase pain sensitivity is thirst perception. Geuter et al. (51) showed that an increase in thirst perception—induced by consumption of salty snacks followed by 5 h of fluid deprivation—led to an increase in pain intensity ratings during noxious heat stimulation, whereas subsequent ad libitum fluid intake, which reduced thirst perception, decreased pain ratings. In another study, stimulation of thirst sensation via hypertonic saline infusion increased pain intensity ratings during noxious pressure stimulation, which were associated with stronger activation of pain-related brain regions; namely, the anterior cingulate cortex and insula (52). Similar findings were reported in the study by Ogino et al. (7). These brain regions were previously shown to be activated and deactivated independently by an increase and decrease in thirst perception, respectively (52–54). However, it is not apparent from these data whether the increased pain sensitivity and pain-related brain activity were due to thirst perception per se, physiological hypohydration, or both.

# Menstrual Phase and Experimental Pain Sensitivity
# Menstrual phase did not modulate the hyperalgesic effect of hypohydration, nor did it independently affect pain sensitivity. The lack of a menstrual phase effect on ischemic pain sensitivity is consistent with the findings of most previous studies that examined ischemic pain responses across the menstrual cycle (18, 55–58). In contrast, studies by Fillingim et al. (59) and Pfleeger et al. (60) observed greater pain sensitivity during the luteal compared with follicular phase. However, concentrations of 17β-estradiol and progesterone were not measured in the study by Pfleeger et al. (60), whereas the specific definition of the luteal phase used by Fillingim et al. (59) differed from that in the current study. We examined the ML phase when concentrations of both 17β-estradiol and progesterone are elevated, whereas Fillingim et al. (59) included the late-luteal period in their classification of the luteal phase, during which concentrations of 17β-estradiol and progesterone are declining rapidly and are less stable compared with the ML phase. Given that menstrual phases characterized by fluctuating or unstable 17β-estradiol levels have been associated with increased severity of various chronic pain conditions (14), this may explain why Fillingim et al. (59) observed phase differences in pain sensitivity whereas we did not.

# The equivocal and inconclusive results regarding menstrual phase effects on experimental pain sensitivity are thought to be due, in part, to the lack of proper verification of menstrual phases through blood measurements of 17β-estradiol and progesterone (15, 16), making it difficult to draw conclusions about the relationship between menstrual phase/hormone status and pain sensitivity. Similar to previous observations (22, 23), Sosm was lower in the ML than EF phase (Table 2); however, this phase difference in hydration did not appear to impact any of the ischemic pain measures (Figs. 1–3). In this study, we showed that after hydration status was controlled, menstrual phases were properly verified and potential confounders (e.g., dietary and lifestyle factors and dysmenorrhea) were minimized, there was no difference in ischemic pain sensitivity between the EF and ML phases. As such, our data suggest that the hormonal fluctuations within the menstrual cycle do not greatly impact pain sensitivity.

# Interestingly, although we did not find differences in pain sensitivity between the EF and ML phase, there was a positive correlation between 17β-estradiol and pain sensitivity, suggesting that women with higher baseline levels of 17β-estradiol may be more sensitive to pain than those with lower 17β-estradiol levels (i.e., between-subject effect). However, given the weak strength of the correlation (r = ∼0.32) (48), this finding should be interpreted with caution.

# Effects of Acute Water Ingestion
# Given that thirst perception may be a potential mechanism linking hypohydration and increased pain sensitivity, we hypothesized that acute water ingestion when hypohydrated would reduce pain sensitivity. This would also help to delineate the effects of thirst and hypohydration per se, on pain, since acute water ingestion would be expected to reduce thirst without altering hydration status (38, 61). However, contrary to our hypothesis, ischemic pain tolerance was not affected by ingestion of 5 mL·kg−1 of water, despite the significant decrease in thirst perception ratings. Meanwhile, ratings of both pain intensity and pain unpleasantness increased, which may appear counterintuitive as participants reported feeling more pain despite being less thirsty. These unexpected findings may be explained by carryover (sensitization) effects from the baseline pain test. A previous study showed that repeated pain stimulation within a single session produced a sensitization effect, whereby pain ratings increased gradually with each repeated stimulation (62). Therefore, this sensitization effect may have masked potential positive effects of acute water ingestion on pain. Future studies that either utilize a longer interval between the repeated pain tests, or perform the postwater ingestion pain test on a separate day, are required to gain further understanding on the relative importance of thirst perception and physiological hypohydration on pain.

# Considerations
# There are several considerations and limitations of this study that should be noted in addition to those already discussed earlier. First, only healthy, eumenorrheic women were included in this study, which ignores postmenopausal women and oral contraceptive pill users, who have different hormonal profiles compared with eumenorrheic women. Therefore, it is possible that hypohydration may affect pain sensitivity differently in these women compared with eumenorrheic women. Second, blood samples were not obtained after water ingestion to confirm that hydration status was not altered. However, previous data indicate that consuming a moderate volume of water, similar to that in the current study, does not acutely alter hydration status in hypohydrated men (38, 61). In addition, the relatively short interval of 30 min between water ingestion and the subsequent pain test was likely insufficient for hydration status to be altered, because a previous study showed that at least 75 min is required for 300 mL of water to be completely absorbed into the bloodstream (63). Third, as participants could not be blinded to the hydration intervention, the increased pain sensitivity observed during HYPO may have been confounded by nocebo effects (64, 65). Although participants were not aware of the study hypotheses, they were aware of the aims of the study, one of which was to investigate the effects of hypohydration on pain sensitivity. Given the negative connotations of hypohydration or dehydration, it is conceivable that participants would expect hypohydration to increase pain sensitivity, which may have subsequently influenced participants’ actual responses to the ischemic pain test. Fourth, the body mass change values in HYPO may not accurately reflect the fluid losses induced by the 24 h of fluid restriction. Due to the scheduling constraints that included two experimental visits within 3 days per menstrual phase, we were unable to obtain a baseline body mass at the start of the hydration intervention; thus, body mass change in HYPO was calculated using EUH body mass. As the EUH and HYPO trials within each menstrual phase were separated by at least 48 h, it is likely that the values were confounded by factors other than the change in body water content (e.g., urine and fecal losses, food and fluid consumption). Acknowledging this limitation, we obtained other measures of hydration status (i.e., Sosm, USG, copeptin, and thirst ratings) in addition to body mass change for a more accurate assessment of hydrations status. Finally, our sample size may have limited our ability to detect statistically significant menstrual phase effects on pain. As such, the effects of menstrual phase on pain require further investigation. Our study could provide a reference for future studies with a similar design to base their sample size calculations on. Nevertheless, large effect sizes (η2p
#  = 0.37–0.55) were obtained for the main effect of hypohydration on pain, with an observed power of 0.7–0.9, which suggests a low probability of false negatives.

# Perspectives and Significance
# Our data indicate that hypohydration increases pain sensitivity in women and this effect may not be reversed immediately by water ingestion. Whereas, menstrual phase (EF vs. ML) does not appear to affect pain sensitivity. These findings suggest that insufficient water intake throughout the day may exacerbate pain perception in healthy women. Given the high prevalence of hypohydration among the general population (25), future studies should investigate whether hypohydration may worsen chronic pain conditions, and the potential efficacy of maintaining adequate hydration in alleviating pain symptoms. This may be especially important for older adults, who are more likely to be hypohydrated and experience chronic pain (25, 66). For human pain researchers, controlling or measuring hydration status of participants may be important to minimize confound, as clinically and physiologically relevant tests such as the cold pressor are affected (67). Although we did not find differences in pain sensitivity between the EF and ML phases after controlling for hydration status, more high-quality research is required to accurately determine whether menstrual phase does or does not affect pain sensitivity. Therefore, until a consensus on the topic has been reached, our data do not discount the need for future pain researchers to verify menstrual phase.

# """
# text_dict

{1: {'title': 'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women',
  'text_discussion': '\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min after handgrip exercise (hereafter referred to as pain ratings) during ischemic pain stimulation, when compared 

In [40]:
text = """Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women

DISCUSSION
This study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.

Hypohydration and Pain in Women
Independent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min after handgrip exercise (hereafter referred to as pain ratings) during ischemic pain stimulation, when compared with euhydration. These findings are consistent with those of previous studies that showed negative effects of mild hypohydration on experimental pain sensitivity in men (7, 8). Ogino et al. (7) observed a decrease in pain thresholds (i.e., increased pain sensitivity) following a combination of exercise-induced dehydration and fluid restriction, whereas Bear et al. (8) found an increase in pain intensity ratings but no difference in pain thresholds after 24 h of fluid restriction. Therefore, the current findings extend these previous results to women by showing that mild hypohydration also increases experimental pain sensitivity in women. The previous studies also did not include pain measures that reflect the affective/emotional experience of pain (i.e., pain tolerance or pain unpleasantness ratings) that may be important for chronic pain, whereas both pain threshold and pain intensity ratings assess the sensory/discriminative aspect of pain (49, 50). Although Bear et al. (8) measured pain tolerance, the nature of the stimulus-response relationship for the cold pressor task—i.e., plateau and gradual decline in the perceived intensity of the pain stimulus (41)—resulted in a large proportion (∼2/3) of participants who were able to tolerate the pain until the 4-min cut-off time, thereby precluding meaningful analyses of the results. This issue was not apparent with the ischemic pain test used in the present study, as there was only one instance where the 20-min time limit was reached. Therefore, the current findings further add to the previous data by showing that mild hypohydration also negatively impacts the affective/emotional aspect of pain, as evidenced by the decrease in pain tolerance and increase in pain unpleasantness ratings compared with EUH.

The exact mechanisms underlying the hyperalgesic effect of hypohydration cannot be determined from the current data. However, one potential mechanism through which hypohydration may increase pain sensitivity is thirst perception. Geuter et al. (51) showed that an increase in thirst perception—induced by consumption of salty snacks followed by 5 h of fluid deprivation—led to an increase in pain intensity ratings during noxious heat stimulation, whereas subsequent ad libitum fluid intake, which reduced thirst perception, decreased pain ratings. In another study, stimulation of thirst sensation via hypertonic saline infusion increased pain intensity ratings during noxious pressure stimulation, which were associated with stronger activation of pain-related brain regions; namely, the anterior cingulate cortex and insula (52). Similar findings were reported in the study by Ogino et al. (7). These brain regions were previously shown to be activated and deactivated independently by an increase and decrease in thirst perception, respectively (52–54). However, it is not apparent from these data whether the increased pain sensitivity and pain-related brain activity were due to thirst perception per se, physiological hypohydration, or both.

Menstrual Phase and Experimental Pain Sensitivity
Menstrual phase did not modulate the hyperalgesic effect of hypohydration, nor did it independently affect pain sensitivity. The lack of a menstrual phase effect on ischemic pain sensitivity is consistent with the findings of most previous studies that examined ischemic pain responses across the menstrual cycle (18, 55–58). In contrast, studies by Fillingim et al. (59) and Pfleeger et al. (60) observed greater pain sensitivity during the luteal compared with follicular phase. However, concentrations of 17β-estradiol and progesterone were not measured in the study by Pfleeger et al. (60), whereas the specific definition of the luteal phase used by Fillingim et al. (59) differed from that in the current study. We examined the ML phase when concentrations of both 17β-estradiol and progesterone are elevated, whereas Fillingim et al. (59) included the late-luteal period in their classification of the luteal phase, during which concentrations of 17β-estradiol and progesterone are declining rapidly and are less stable compared with the ML phase. Given that menstrual phases characterized by fluctuating or unstable 17β-estradiol levels have been associated with increased severity of various chronic pain conditions (14), this may explain why Fillingim et al. (59) observed phase differences in pain sensitivity whereas we did not.

The equivocal and inconclusive results regarding menstrual phase effects on experimental pain sensitivity are thought to be due, in part, to the lack of proper verification of menstrual phases through blood measurements of 17β-estradiol and progesterone (15, 16), making it difficult to draw conclusions about the relationship between menstrual phase/hormone status and pain sensitivity. Similar to previous observations (22, 23), Sosm was lower in the ML than EF phase (Table 2); however, this phase difference in hydration did not appear to impact any of the ischemic pain measures (Figs. 1–3). In this study, we showed that after hydration status was controlled, menstrual phases were properly verified and potential confounders (e.g., dietary and lifestyle factors and dysmenorrhea) were minimized, there was no difference in ischemic pain sensitivity between the EF and ML phases. As such, our data suggest that the hormonal fluctuations within the menstrual cycle do not greatly impact pain sensitivity.

Interestingly, although we did not find differences in pain sensitivity between the EF and ML phase, there was a positive correlation between 17β-estradiol and pain sensitivity, suggesting that women with higher baseline levels of 17β-estradiol may be more sensitive to pain than those with lower 17β-estradiol levels (i.e., between-subject effect). However, given the weak strength of the correlation (r = ∼0.32) (48), this finding should be interpreted with caution.

Effects of Acute Water Ingestion
Given that thirst perception may be a potential mechanism linking hypohydration and increased pain sensitivity, we hypothesized that acute water ingestion when hypohydrated would reduce pain sensitivity. This would also help to delineate the effects of thirst and hypohydration per se, on pain, since acute water ingestion would be expected to reduce thirst without altering hydration status (38, 61). However, contrary to our hypothesis, ischemic pain tolerance was not affected by ingestion of 5 mL·kg−1 of water, despite the significant decrease in thirst perception ratings. Meanwhile, ratings of both pain intensity and pain unpleasantness increased, which may appear counterintuitive as participants reported feeling more pain despite being less thirsty. These unexpected findings may be explained by carryover (sensitization) effects from the baseline pain test. A previous study showed that repeated pain stimulation within a single session produced a sensitization effect, whereby pain ratings increased gradually with each repeated stimulation (62). Therefore, this sensitization effect may have masked potential positive effects of acute water ingestion on pain. Future studies that either utilize a longer interval between the repeated pain tests, or perform the postwater ingestion pain test on a separate day, are required to gain further understanding on the relative importance of thirst perception and physiological hypohydration on pain.

Considerations
There are several considerations and limitations of this study that should be noted in addition to those already discussed earlier. First, only healthy, eumenorrheic women were included in this study, which ignores postmenopausal women and oral contraceptive pill users, who have different hormonal profiles compared with eumenorrheic women. Therefore, it is possible that hypohydration may affect pain sensitivity differently in these women compared with eumenorrheic women. Second, blood samples were not obtained after water ingestion to confirm that hydration status was not altered. However, previous data indicate that consuming a moderate volume of water, similar to that in the current study, does not acutely alter hydration status in hypohydrated men (38, 61). In addition, the relatively short interval of 30 min between water ingestion and the subsequent pain test was likely insufficient for hydration status to be altered, because a previous study showed that at least 75 min is required for 300 mL of water to be completely absorbed into the bloodstream (63). Third, as participants could not be blinded to the hydration intervention, the increased pain sensitivity observed during HYPO may have been confounded by nocebo effects (64, 65). Although participants were not aware of the study hypotheses, they were aware of the aims of the study, one of which was to investigate the effects of hypohydration on pain sensitivity. Given the negative connotations of hypohydration or dehydration, it is conceivable that participants would expect hypohydration to increase pain sensitivity, which may have subsequently influenced participants’ actual responses to the ischemic pain test. Fourth, the body mass change values in HYPO may not accurately reflect the fluid losses induced by the 24 h of fluid restriction. Due to the scheduling constraints that included two experimental visits within 3 days per menstrual phase, we were unable to obtain a baseline body mass at the start of the hydration intervention; thus, body mass change in HYPO was calculated using EUH body mass. As the EUH and HYPO trials within each menstrual phase were separated by at least 48 h, it is likely that the values were confounded by factors other than the change in body water content (e.g., urine and fecal losses, food and fluid consumption). Acknowledging this limitation, we obtained other measures of hydration status (i.e., Sosm, USG, copeptin, and thirst ratings) in addition to body mass change for a more accurate assessment of hydrations status. Finally, our sample size may have limited our ability to detect statistically significant menstrual phase effects on pain. As such, the effects of menstrual phase on pain require further investigation. Our study could provide a reference for future studies with a similar design to base their sample size calculations on. Nevertheless, large effect sizes (η2p
 = 0.37–0.55) were obtained for the main effect of hypohydration on pain, with an observed power of 0.7–0.9, which suggests a low probability of false negatives.

Perspectives and Significance
Our data indicate that hypohydration increases pain sensitivity in women and this effect may not be reversed immediately by water ingestion. Whereas, menstrual phase (EF vs. ML) does not appear to affect pain sensitivity. These findings suggest that insufficient water intake throughout the day may exacerbate pain perception in healthy women. Given the high prevalence of hypohydration among the general population (25), future studies should investigate whether hypohydration may worsen chronic pain conditions, and the potential efficacy of maintaining adequate hydration in alleviating pain symptoms. This may be especially important for older adults, who are more likely to be hypohydrated and experience chronic pain (25, 66). For human pain researchers, controlling or measuring hydration status of participants may be important to minimize confound, as clinically and physiologically relevant tests such as the cold pressor are affected (67). Although we did not find differences in pain sensitivity between the EF and ML phases after controlling for hydration status, more high-quality research is required to accurately determine whether menstrual phase does or does not affect pain sensitivity. Therefore, until a consensus on the topic has been reached, our data do not discount the need for future pain researchers to verify menstrual phase.

"""
text

'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min after handgrip exercise (hereafter referred to as pain ratings) during ischemic pain stimulation, when compared with euhydration. These findings are 

In [44]:
pattern = r'\s?(\S*)(\n*)(.+)'
re.sub(pattern, r'\1', text)
re.match(pattern, text)[0]

'Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women'

In [13]:
test_text

'\nHypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\nHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenDHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenIHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenSHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenCHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenUHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenSHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenSHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenIHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenOHypohydration but not Menstrual Phase Influences Pain Perception in Healthy WomenNHypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\nHypohydrati

# `Summarize`

In [70]:


class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices')
        openai.api_key = os.getenv('api_openai')
        try:
            response = openai.ChatCompletion.create(
                model=self.model, messages=messages, 
                temperature=self.temperature, 
                max_tokens=self.max_tokens,
                n=self.n_choices
                )
            print('Done sending request to GPT-3')
        except:
            print('Request error')
        return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1500,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    try:
        response = chatbot.gpt(prompt)
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
        return qna, chatbot
    except:
        print('**Check response object**')
        return prompt, chatbot
    
qna_dict = {}
chatbot_dict = {}
iteration = 1
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
qna_dict[1]


Done creating prompt
[{'role': 'system', 'content': 'You are an expert at science communication.'}, {'role': 'user', 'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1

{1: [{'role': 'system',
   'content': 'You are an expert at science communication.'},
  {'role': 'user',
   'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min afte

In [77]:
qna_dict[1][1]

[{'role': 'system', 'content': 'You are an expert at science communication.'},
 {'role': 'user',
  'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1 min after handgri

In [79]:
response = openai.ChatCompletion.create(
                messages=qna_dict[1][1], model='gpt-3.5-turbo'
                )
response

<OpenAIObject chat.completion id=chatcmpl-72SLKdwUtB57YzYBq8Mx5FyEZwq1l at 0x19801826e50> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "This study found that not drinking enough water increases pain sensitivity in healthy women, but menstrual cycle has no effect on pain. Acute water intake did not immediately reduce pain sensitivity. The findings suggest that drinking enough water throughout the day may help alleviate pain symptoms in women. Future studies will be needed to investigate whether insufficient water intake worsens chronic pain conditions. The study also emphasizes the importance of controlling or measuring hydration status in pain research.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680819962,
  "id": "chatcmpl-72SLKdwUtB57YzYBq8Mx5FyEZwq1l",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 86,
    "prompt_tokens": 2602,
    "total

In [56]:
combinations

,task part 1,task part 2,audience,prompt
0,Summarize the article,,lay audience,Summarize the article for lay audience.
1,Summarize the article,,,Summarize the article .


# Iteration2

In [86]:


class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response
        # try:
        #     response = openai.ChatCompletion.create(
        #         model=self.model, messages=messages, 
        #         temperature=self.temperature, 
        #         max_tokens=self.max_tokens,
        #         n=self.n_choices
        #         )
        #     print('Done sending request to GPT-3')
        # except:
        #     print('Request error')
        # return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1500,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    return chatbot.gpt(prompt), chatbot
    # try:
    #     response = chatbot.gpt(prompt)
    #     for index, choice in enumerate(response.choices):
    #         qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
    #     for index, source in enumerate(sources):
    #         qna[f'source_{index+1}'] = source
    #     return qna, chatbot
    # except:
    #     print('**Check response object**')
    #     return prompt, chatbot
    

iteration = 2
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
qna_dict[2]


Done creating prompt
[{'role': 'system', 'content': 'You are an expert at science communication.'}, {'role': 'user', 'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1

InvalidRequestError: This model's maximum context length is 4096 tokens. However, you requested 4102 tokens (2602 in the messages, 1500 in the completion). Please reduce the length of the messages or completion.

In [88]:
iteration = 2.2
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text, max_tokens=1000)
    qna_dict[iteration][prompt_num] = response
qna_dict[2.2]


Done creating prompt
[{'role': 'system', 'content': 'You are an expert at science communication.'}, {'role': 'user', 'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1

{1: <OpenAIObject chat.completion id=chatcmpl-72SjH7cX2awWm4Qfnfwa0NQp9L5xo at 0x19801825220> JSON: {
   "choices": [
     {
       "finish_reason": "stop",
       "index": 0,
       "message": {
         "content": "Dehydration can increase pain sensitivity in women, according to a new study. The research, which involved healthy women of reproductive age, found that even mild dehydration can reduce pain tolerance and increase pain intensity and unpleasantness. Menstrual cycle phase did not affect pain sensitivity, and drinking water did not alleviate pain. The study suggests that maintaining adequate hydration may be important in managing chronic pain conditions, particularly among older adults who are more likely to be dehydrated.",
         "role": "assistant"
       }
     },
     {
       "finish_reason": "stop",
       "index": 1,
       "message": {
         "content": "A recent study found that mild dehydration can increase pain sensitivity in healthy women. The study looked at

In [97]:
qna_dict[2.2][1].choices[0]["message"]["content"]

'Dehydration can increase pain sensitivity in women, according to a new study. The research, which involved healthy women of reproductive age, found that even mild dehydration can reduce pain tolerance and increase pain intensity and unpleasantness. Menstrual cycle phase did not affect pain sensitivity, and drinking water did not alleviate pain. The study suggests that maintaining adequate hydration may be important in managing chronic pain conditions, particularly among older adults who are more likely to be dehydrated.'

# Iteration 3

In [89]:
class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response
        # try:
        #     response = openai.ChatCompletion.create(
        #         model=self.model, messages=messages, 
        #         temperature=self.temperature, 
        #         max_tokens=self.max_tokens,
        #         n=self.n_choices
        #         )
        #     print('Done sending request to GPT-3')
        # except:
        #     print('Request error')
        # return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    try:
        response = chatbot.gpt(prompt)
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
        return qna, chatbot
    except:
        print('**API request failed**')
        return prompt, chatbot
    

iteration = 3
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
# pd.DataFrame(qna_dict[3])


Done creating prompt
[{'role': 'system', 'content': 'You are an expert at science communication.'}, {'role': 'user', 'content': 'Given the following text: Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women\n\nDISCUSSION\nThis study examined the independent and combined effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy to the deleterious impact of hypohydration. The main findings were that: 1) mild hypohydration increased pain sensitivity, 2) menstrual phase did not affect pain sensitivity, nor did it influence the effect of hypohydration on pain, and 3) acute water ingestion did not reduce pain sensitivity.\n\nHypohydration and Pain in Women\nIndependent of menstrual phase, mild hypohydration following 24 h of fluid restriction reduced pain tolerance (−8%) and increased pain intensity (+13%) and pain unpleasantness (+12%) ratings at 1

# Iteration 4

In [99]:
class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        # print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response
        # try:
        #     response = openai.ChatCompletion.create(
        #         model=self.model, messages=messages, 
        #         temperature=self.temperature, 
        #         max_tokens=self.max_tokens,
        #         n=self.n_choices
        #         )
        #     print('Done sending request to GPT-3')
        # except:
        #     print('Request error')
        # return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    response = chatbot.gpt(prompt)
    
    try:
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        for index, source in enumerate(sources):
            qna[f'source_{index+1}'] = source
        return qna, chatbot
    except:
        print('**API request failed**')
        return response, chatbot
    

iteration = 4
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
# pd.DataFrame(qna_dict[3])


Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3
**API request failed**
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3
**API request failed**


In [105]:
qna = dict()
for index, choice in enumerate(response.choices):
    qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
qna
    

{'response_01': 'The study examined the effects of hypohydration and menstrual phase on pain sensitivity in healthy women. The findings show that hypohydration increases pain sensitivity in women, but menstrual phase does not have an independent effect on pain sensitivity. Acute water ingestion did not reduce pain sensitivity, indicating that the effect of hypohydration on pain may not be immediately reversible. The study suggests that maintaining adequate hydration may be important for alleviating pain symptoms, especially in older adults who are more likely to be hypohydrated and experience chronic pain. The article also discusses the limitations of the study and the need for further research to accurately determine the effects of menstrual phase on pain sensitivity.',
 'response_02': 'The study investigated the effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy eumenorrheic women, and the potential efficacy of acute water ingestion as a remedy t

# Iteration 5

In [106]:
class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        # print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response
        # try:
        #     response = openai.ChatCompletion.create(
        #         model=self.model, messages=messages, 
        #         temperature=self.temperature, 
        #         max_tokens=self.max_tokens,
        #         n=self.n_choices
        #         )
        #     print('Done sending request to GPT-3')
        # except:
        #     print('Request error')
        # return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    try:
        response = chatbot.gpt(prompt)
    except:
        print('**API request failed**')
        return qna, chatbot
    try:
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
            qna[f'text'] = text
        return qna, chatbot
    except:
        print('**Error with response parsing**')
        return qna, response
    

iteration = 5
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
# pd.DataFrame(qna_dict[5])


Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3


## Results to save

In [110]:
pd.DataFrame(qna_dict[5]).drop('text')

,1,2
article_title,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women,Hypohydration but not Menstrual Phase Influences Pain Perception in Healthy Women
system_role,You are an expert at science communication.,You are an expert at science communication.
prompt,Summarize the article for lay audience.,Summarize the article .
model,gpt-3.5-turbo,gpt-3.5-turbo
response_01,"A new study has found that mild dehydration can increase pain sensitivity in healthy women. The research examined the effects of dehydration and menstrual cycle on pain perception in women, and found that dehydration caused an increase in pain intensity and unpleasantness. However, the study also found that menstrual cycle did not have a significant effect on pain sensitivity, nor did drinking water help to alleviate the pain caused by dehydration. These findings suggest that staying hydrated throughout the day may be important in reducing pain perception, particularly for those who experience chronic pain. The study also highlights the need for researchers to control or measure hydration status when studying pain.","The study examined the effects of hypohydration and menstrual phase on pain sensitivity in healthy women. The findings showed that mild hypohydration increased pain sensitivity, but menstrual phase did not affect pain sensitivity. Acute water ingestion did not reduce pain sensitivity. The study suggests that insufficient water intake may worsen pain perception in healthy women and highlights the need for future studies to investigate the potential efficacy of maintaining adequate hydration in alleviating pain symptoms. The study also suggests that controlling or measuring hydration status of participants may be important for pain researchers to minimize confound."
response_02,"This study found that not drinking enough water can increase pain sensitivity in healthy women. Menstrual phase did not affect pain sensitivity, but higher levels of the hormone 17β-estradiol were associated with more pain. Drinking water did not immediately reduce pain sensitivity. The study suggests that staying hydrated may help alleviate pain symptoms and more research is needed to determine how dehydration affects chronic pain conditions.","The article reports on a study that investigated the effects of hypohydration and menstrual phase on experimental pain sensitivity in healthy women. The study found that mild hypohydration increased pain sensitivity, while menstrual phase did not have any effect on pain sensitivity. Acute water ingestion did not reduce pain sensitivity, despite reducing thirst perception. The findings suggest that insufficient water intake may exacerbate pain perception in healthy women, and future studies should investigate whether hypohydration worsens chronic pain conditions. The study also highlights the importance of controlling or measuring hydration status in pain research."
response_03,"A new study has found that not drinking enough water can increase pain sensitivity in healthy women, regardless of their menstrual cycle phase. The research involved mild dehydration in healthy women for 24 hours, which led to increased pain intensity and unpleasantness ratings. The study also found that drinking water did not reduce pain sensitivity, despite reducing thirst. The researchers suggest that future studies should investigate whether dehydration could worsen chronic pain conditions and the potential benefits of maintaining adequate hydration in alleviating pain symptoms.","The study examined the effects of hypohydration and menstrual phase on pain sensitivity in healthy women. The results showed that mild hypohydration increased pain sensitivity, while menstrual phase did not affect pain sensitivity. Acute water ingestion did not reduce pain sensitivity. The study suggests that insufficient water intake may exacerbate pain perception in healthy women and future studies should investigate whether hypohydration worsens chro

# Iteration 6

In [ ]:
class Chatbot:
    def __init__(self, text, system_role, model, temperature, n_choices, max_tokens):
        self.text = text
        self.system_role = system_role
        self.temperature = temperature
        self.n_choices = n_choices
        self.max_tokens = max_tokens
        self.model = model
    
    def create_prompt(self, task):
        system_role = f'{self.system_role}'
        user_input = f"""Given the following text: {self.text} \n {task}"""
        messages = [
        {"role": "system", "content": system_role},
        {"role": "user", "content": user_input},]

        print('Done creating prompt')
        # print(messages)
        return messages

    def gpt(self, messages):
        print('Sending request to GPT-3')
        print(f'\tRequesting {self.n_choices} choices using {self.model}')
        openai.api_key = os.getenv('api_openai')
        response = openai.ChatCompletion.create(
            model=self.model, messages=messages, 
            temperature=self.temperature, 
            max_tokens=self.max_tokens,
            n=self.n_choices
            )
        print('Done sending request to GPT-3')
        return response


def reply(task, text, model="gpt-3.5-turbo", temperature=0.7, n_choices=5, max_tokens=1000,
          text_key=['text_discussion'],
        system_role = "You are an expert at science communication."
        ):
    chatbot = Chatbot(text,
        system_role=system_role, model=model, temperature=temperature, n_choices=n_choices,
        max_tokens=max_tokens
        )
    prompt = chatbot.create_prompt(task)
    firstline_pattern = r'\s?(\S*)(\n*)(.+)'
    title = re.match(firstline_pattern, text)[0]
    qna = dict()
    qna['article_title'] = title
    qna['system_role'] = system_role
    qna['prompt'] = task
    qna['model'] = model
    try:
        response = chatbot.gpt(prompt)
    except:
        print('**API request failed**')
        return qna, chatbot
    try:
        for index, choice in enumerate(response.choices):
            qna[f'response_{"{:02d}".format(index+1)}'] = choice["message"]["content"]
        qna[f'text'] = text
        return qna, chatbot
    except:
        print('**Error with response parsing**')
        return qna, response
    

iteration = 6
qna_dict[iteration] = dict()

for index, input in combinations['prompt'].items():
    # print(input)
    prompt_num = index+1
    response, chatbot_dict[f'{iteration}_prompt{prompt_num}'] = reply(input, text)
    qna_dict[iteration][prompt_num] = response
# pd.DataFrame(qna_dict[5])


Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3
Done creating prompt
Sending request to GPT-3
	Requesting 5 choices using gpt-3.5-turbo
Done sending request to GPT-3


# Test with prompt variants

# *End of Page*